# Pipeline Baseline: YOLO → Pinhole → ResNet → 6D Pose
Questa versione minimale esegue la pipeline base per la stima della posa 6D su LineMOD, ispirata al notebook test5_baseline_pinhole_pipeline.ipynb.

**Nota:** Questa pipeline minimale esegue solo un batch di test e mostra la traduzione e la rotazione predette. Per valutazioni complete e metriche, vedere il notebook originale test5_baseline_pinhole_pipeline.ipynb.

## 1. Import essenziali e setup minimal

In [1]:
# Import essenziali e setup minimal
import sys
from pathlib import Path
import torch
import os
import pandas as pd

sys.path.insert(0, str(Path.cwd().parent))
from config import Config
from models.pose_estimator_baseline import PoseEstimatorBaseline
from models.yolo_detector import YOLODetector
from utils.pinhole import compute_translation_pinhole
from dataset.linemod_pose import create_pose_dataloaders
from utils.validation import load_validation_results, calc_add_accuracy_per_class, run_yolo_baseline_pipeline
from utils.visualization import plot_add_per_class


## 2. Carica modelli pre-addestrati (YOLO e PoseEstimatorBaseline)

In [2]:
train_loader, val_loader, test_loader = create_pose_dataloaders(
    dataset_root=Config.LINEMOD_ROOT,
    batch_size=Config.POSE_BATCH_SIZE,
    crop_margin=Config.POSE_CROP_MARGIN,
    output_size=Config.POSE_IMAGE_SIZE,
    num_workers=Config.NUM_WORKERS_POSE
)

In [3]:
# Carica modelli pre-addestrati (YOLO e PoseEstimatorBaseline)
yolo_weights = Config.CHECKPOINT_DIR / 'yolo' / 'yolo_train10' / 'weights' / 'best.pt'
yolo_detector = YOLODetector(model_name=str(yolo_weights), num_classes=Config.NUM_CLASSES) if yolo_weights.exists() else None
POSE_MODEL_NAME = 'pose_rgb_baseline'
pose_ckpt = Config.CHECKPOINT_DIR / 'pose' / POSE_MODEL_NAME / 'weights' / 'best.pt'
model_baseline = PoseEstimatorBaseline(pretrained=True).to(Config.DEVICE)
if pose_ckpt.exists():
    model_baseline.load_state_dict(torch.load(pose_ckpt, map_location=Config.DEVICE))
model_baseline.eval()

✅ Loading custom weights from: /Users/nicolotermine/zMellow/GitHub-Poli/Polito/polito-aml-6D_pose_estimation/checkpoints/yolo/yolo_train10/weights/best.pt
PoseEstimatorBaseline initialized (BASELINE MODEL)
   Backbone: resnet50 (weights=ResNet50_Weights.IMAGENET1K_V1, frozen=False)
   Feature dim: 2048
   Output: 4 values (quaternion only)
   Translation: Computed with Pinhole Camera Model (NOT learned)
   Dropout: 0.5


✅ Loading custom weights from: /Users/nicolotermine/zMellow/GitHub-Poli/Polito/polito-aml-6D_pose_estimation/checkpoints/yolo/yolo_train10/weights/best.pt
PoseEstimatorBaseline initialized (BASELINE MODEL)
   Backbone: resnet50 (weights=ResNet50_Weights.IMAGENET1K_V1, frozen=False)
   Feature dim: 2048
   Output: 4 values (quaternion only)
   Translation: Computed with Pinhole Camera Model (NOT learned)
   Dropout: 0.5


PoseEstimatorBaseline(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (

## 3. Pipeline completa YOLO + ResNet + Pinhole

Valuta la pipeline baseline end-to-end con YOLO detection su immagini full-size.  
Puoi scegliere tra:
- **Debug veloce**: 10 immagini (cella 4.1)
- **Validazione completa**: tutto il test set (cella 4.2)

In [ ]:
# 🐛 DEBUG VELOCE: Test pipeline YOLO su solo 10 immagini
train_loader, val_loader, test_loader = create_pose_dataloaders(
    dataset_root=Config.LINEMOD_ROOT,
    batch_size=Config.POSE_BATCH_SIZE,
    crop_margin=Config.POSE_CROP_MARGIN,
    output_size=Config.POSE_IMAGE_SIZE,
    num_workers=Config.NUM_WORKERS_POSE
)

# ✅ Usa max_samples=10 per debug veloce
run_yolo_baseline_pipeline(
    yolo_detector, 
    model_baseline, 
    test_loader, 
    name=POSE_MODEL_NAME,
    max_samples=10  # 🔧 Solo 10 immagini per debug rapido
)

print("\n💡 Per la validazione completa su tutto il test set, esegui la cella successiva!")

In [ ]:
# 🚀 VALIDAZIONE COMPLETA: Esegui la pipeline baseline su TUTTO il test set
# ⚠️ Questa cella può richiedere diversi minuti. Per debug veloce usa la cella precedente (10 immagini)

train_loader, val_loader, test_loader = create_pose_dataloaders(
    dataset_root=Config.LINEMOD_ROOT,
    batch_size=Config.POSE_BATCH_SIZE,
    crop_margin=Config.POSE_CROP_MARGIN,
    output_size=Config.POSE_IMAGE_SIZE,
    num_workers=Config.NUM_WORKERS_POSE
)

# Processa tutto il test set (nessun limite)
run_yolo_baseline_pipeline(yolo_detector, model_baseline, test_loader, name=POSE_MODEL_NAME)

## 4. Carica e mostra tabella delle metriche per classe (ADD full pose)

In [ ]:
# Carica e mostra tabella delle metriche per classe (ADD full pose)
val_csv_path = os.path.join(Config.CHECKPOINT_DIR, 'pose', POSE_MODEL_NAME, 'validation_result.csv')
results_full_pose, _ = load_validation_results(val_csv_path)

data, global_add, global_acc = calc_add_accuracy_per_class(results_full_pose, Config.LINEMOD_OBJECTS)

df = pd.DataFrame(data)
display(df)
print(f"\nMedia globale ADD (full pose): {global_add:.2f}")
print(f"Accuracy globale (full pose) (%): {global_acc:.1f}")

## 5. Grafico a barre delle medie ADD per classe (rot-only)
Mostra le medie della metrica ADD per ciascuna classe oggetto, calcolate sulla rotazione predetta.

In [ ]:
# Grafico a barre delle medie ADD per classe (full pose end-to-end)
results_full_pose = globals().get('results_full_pose', None)
if results_full_pose is None:
    print("⚠️  Devi prima calcolare la metrica ADD full pose su tutto il test set e salvare i risultati in 'results_full_pose'.")
else:
    plot_add_per_class(results_full_pose, Config.LINEMOD_OBJECTS)